### Drive Set Up

In [ ]:
#Mount the RAW session level data: 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# The path below should point to the directory containing this notebook and the associated utility files
# Change it if necessary
os.chdir('/content/drive/MyDrive/GroupBy/')

### Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import datetime
from time import strftime
from time import gmtime
import holidays

### Import Data

In [ ]:
#load the 'eCommerce Events History in Cosmetics Shop' data set 
temp_df = pd.read_csv('df_ec_final_0.csv')

In [ ]:
# reset
df_ec_coded = temp_df

### Explore Data

In [ ]:
#preview data
df_ec_coded.head(5)

,event_time,product_id,category_id,product_brand,product_price,customerVisitorId,customerSessionId,unique_key,unique_sess_event,unique_sess,totals_events,event_type_original,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,totalViewProducts,totalViewProductRevenue,totalAddToCarts,totalAddToCartRevenue,totalRemoveFromCarts,totalRemoveFromCartRevenue,totalOrders,totalOrderRevenue,uniqueViewProducts,uniqueAddToCarts,uniqueRemoveFromCarts,uniqueOrders
0,2020-02-29 23:59:59,5927709,1487580013950664926,de.lux,3.17,621669991,daa3c677-5360-4327-847e-d29658716b99,daa3c677-5360-4327-847e-d29658716b99view2020-0...,view2020-02-29 23:59:59,621669991daa3c677-5360-4327-847e-d29658716b99,0.0,view,0,0,0,1,44.0,43.81,10.0,8.10,10.0,9.77,0.0,0.0,23.0,9.0,5.0,0.0
1,2020-02-29 23:59:54,5716351,1487580010872045658,irisk,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d,18af673b-7fb9-4202-a66d-5c855bc0fd2dview2020-0...,view2020-02-29 23:59:54,61984124218af673b-7fb9-4202-a66d-5c855bc0fd2d,0.0,view,0,0,0,1,2.0,1.58,0.0,0.00,0.0,0.00,0.0,0.0,2.0,0.0,0.0,0.0
2,2020-02-29 23:59:45,5850628,1602943681873052386,grattol,5.24,622090043,ab7d349f-db5d-4790-8ab1-31e5c894459d,ab7d349f-db5d-4790-8ab1-31e5c894459dview2020-0...,view2020-02-29 23:59:45,622090043ab7d349f-db5d-4790-8ab1-31e5c894459d,0.0,view,0,0,0,1,2.0,10.00,1.0,5.24,0.0,0.00,0.0,0.0,2.0,1.0,0.0,0.0
3,2020-02-29 23:59:39,5550686,1487580008145748965,roubloff,1.11,459705611,05d2add3-01f7-47ee-8364-27341673227f,05d2add3-01f7-47ee-8364-27341673227fcart2020-0...,cart2020-02-29 23:59:39,45970561105d2add3-01f7-47ee-8364-27341673227f,0.0,cart,1,0,0,0,1.0,1.11,1.0,1.11,0.0,0.00,0.0,0.0,1.0,1.0,0.0,0.0
4,2020-02-29 23:59:32,5885416,1487580005092295511,grattol,6.27,622082947,fb29909b-6ef5-4662-b4ee-288e73e5dc10,fb29909b-6ef5-4662-b4ee-288e73e5dc10view2020-0...,view2020-02-29 23:59:32,622082947fb29909b-6ef5-4662-b4ee-288e73e5dc10,0.0,view,0,0,0,1,9.0,34.38,35.0,117.27,0.0,0.00,0.0,0.0,9.0,30.0,0.0,0.0


In [ ]:
# check missing values to determine next teps
null_columns= df_ec_coded.columns[df_ec_coded.isnull().any()]
df_ec_coded[null_columns].isnull().sum()

Series([], dtype: float64)

In [ ]:
#check percentage of missing values
percent_missing = round(df_ec_coded.isnull().sum() * 100 / len(df_ec_coded),2)
percent_missing

event_time                     0.0
product_id                     0.0
category_id                    0.0
product_brand                  0.0
product_price                  0.0
customerVisitorId              0.0
customerSessionId              0.0
unique_key                     0.0
unique_sess_event              0.0
unique_sess                    0.0
totals_events                  0.0
event_type_original            0.0
event_type_cart                0.0
event_type_purchase            0.0
event_type_remove_from_cart    0.0
event_type_view                0.0
totalViewProducts              0.0
totalViewProductRevenue        0.0
totalAddToCarts                0.0
totalAddToCartRevenue          0.0
totalRemoveFromCarts           0.0
totalRemoveFromCartRevenue     0.0
totalOrders                    0.0
totalOrderRevenue              0.0
uniqueViewProducts             0.0
uniqueAddToCarts               0.0
uniqueRemoveFromCarts          0.0
uniqueOrders                   0.0
dtype: float64

### Create the Time Based Features

In [ ]:
#convert event_time columns to date_time data type
df_ec_coded['event_time'] = df_ec_coded['event_time'].apply(lambda x : x[:-4])
df_ec_coded['event_time'] = pd.to_datetime(df_ec_coded['event_time'])

In [ ]:
#create feature: year
df_ec_coded['event_year'] = df_ec_coded.event_time.dt.year

#create feature: month
df_ec_coded['event_month'] = df_ec_coded.event_time.dt.month

#create feature: day of the week
df_ec_coded['event_day_of_week'] = df_ec_coded.event_time.dt.dayofweek

#create feature: day of the month
df_ec_coded['event_day_of_month'] = df_ec_coded.event_time.dt.day

#create feature: day of the year
df_ec_coded['event_day_of_year'] = df_ec_coded.event_time.dt.dayofyear

#create feature: hour
df_ec_coded['event_hour'] = df_ec_coded.event_time.dt.hour

#create feature: minute
df_ec_coded['event_minute'] = df_ec_coded.event_time.dt.minute

#create feature: second
df_ec_coded['event_second'] = df_ec_coded.event_time.dt.second

#create feature: micorsecond
df_ec_coded['event_micorsecond'] = df_ec_coded.event_time.dt.microsecond

In [ ]:
#create feature: day name
#df_ec_coded['_'] = df_ec_coded.event_time.dt.day_name()
#df_ec_coded = pd.get_dummies(df_ec_coded,columns=['_'])

In [ ]:
#create feature: weekend flag
day_names = df_ec_coded.event_time.dt.day_name()
df_ec_coded['weekend']  = day_names.apply(lambda x : 1 if x in ['Saturday','Sunday'] else 0)

In [ ]:
#create feature: holiday
import holidays
us_holidays = holidays.US()
df_ec_coded['holiday']  = df_ec_coded['event_time'].apply(lambda x : x in us_holidays)

In [ ]:
#re-create feature: sessionStartTime
df_ec_coded = df_ec_coded.join(df_ec_coded.groupby('customerSessionId')['event_time'].agg(['min']), on ='customerSessionId')

#re-create feature: sessionEndTime
df_ec_coded = df_ec_coded.join(df_ec_coded.groupby('customerSessionId')['event_time'].agg(['max']), on ='customerSessionId')

In [ ]:
#re-create feature: timeOnSite
import time
df_ec_coded['timeOnSite'] = df_ec_coded['max']-df_ec_coded['min']
df_ec_coded['timeOnSite'] = df_ec_coded['timeOnSite']/np.timedelta64(1,'s')
#df_ec_coded['timeOnSite'] = df_ec_coded['timeOnSite'].apply(lambda x : time.strftime("%H:%M:%S", time.gmtime(x)))

### Recreate feature for Session meta data

In [ ]:
### re-create feature: newSession  ###

#create dataframe with with no duplicate user_id records and set 'newSession' to True
df_user_id = df_ec_coded.drop_duplicates(subset=['customerVisitorId'], keep='last')
df_user_id['newSession'] = 1

# create a dictionary for reference mapping that uses the category_id column as key and the brand column as values
new_sess_dict = pd.Series(df_user_id.newSession.values,index=df_user_id.customerSessionId).to_dict()

#fill the new_session column using reference mapping 
df_ec_coded['newSession'] = np.nan
df_ec_coded.newSession = df_ec_coded.newSession.fillna(df_ec_coded.customerSessionId.map(new_sess_dict))

#fill the rest of the new_session values with False
df_ec_coded.newSession = df_ec_coded.newSession.fillna(0)

In [ ]:
### re-create feature: hit.isEntrance ###

#create dataframe that contains only the most recent event for each customerSessionId
df_user_sesss = df_ec_coded.drop_duplicates(subset=['customerSessionId'], keep='last')
#add column 'isEntrance' column and set value to True for the first event in the customerSessionId
df_user_sesss['isEntrance'] = 1

#create a unique key for dictionary reference mapping
df_user_sesss['unique_key'] = df_user_sesss['customerSessionId'].map(str)  + df_user_sesss['event_type_original'].map(str)  + df_user_sesss['event_time'].map(str)
# create a dictionary for reference mapping that uses the  column 'unique_key' as the key and the 'isEntrance' column as the values
entry_event = pd.Series(df_user_sesss.isEntrance.values, index = df_user_sesss.unique_key).to_dict()

#add an empty 'isEntrance' column 
df_ec_coded['isEntrance'] = np.nan
#fill the hit.isEntrance column using reference mapping 
df_ec_coded.isEntrance = df_ec_coded.isEntrance.fillna(df_ec_coded.unique_key.map(entry_event))
#fill the rest of the hit.isEntrance values with False
df_ec_coded.isEntrance = df_ec_coded.isEntrance.fillna(0)

In [ ]:
### re-create feature: hit.isExit ###

#create dataframe that contains only the most recent event for each customerSessionId
df_user_sesss_2 = df_ec_coded.drop_duplicates(subset=['customerSessionId'], keep='first')
#add column 'isExit' column and set value to True for the last event in the customerSessionId
df_user_sesss_2['isExit'] = 1

#create a unique key for dictionary reference mapping
df_user_sesss_2['unique_key'] = df_user_sesss_2['customerSessionId'].map(str)  + df_user_sesss_2['event_type_original'].map(str)  + df_user_sesss_2['event_time'].map(str)
# create a dictionary for reference mapping that uses the  column 'unique_key_exit' as the key and the 'isExit' column as the values
exit_event = pd.Series(df_user_sesss_2.isExit.values,index=df_user_sesss_2.unique_key).to_dict()

#add an empty 'isExit' column 
df_ec_coded['isExit'] = np.nan
#fill the hit.isExit column using reference mapping 
df_ec_coded.isExit = df_ec_coded.isExit.fillna(df_ec_coded.unique_key.map(exit_event))
#fill the rest of the hit.isExit values with False
df_ec_coded.isExit = df_ec_coded.isExit.fillna(0)

In [ ]:
### re-create feature: hitSequence ###
df_ec_coded = df_ec_coded.sort_values('event_time',ascending=True)

#create groupby dataframe
df_ec_coded['hits.isEntrance'] = df_ec_coded.groupby(['customerSessionId','customerSessionId']).cumcount()+1

In [ ]:
### re-create feature: customerSessionNumber ###

#create empty 'ccustomerSessionNumber' column
df_ec_coded['customerSessionNumber'] = np.nan

#create a dataframe without duplicate customerSessionId
df_unqiue_sess = df_ec_coded.drop_duplicates(subset='customerSessionId', keep="first")

#create sequence count for customerSessionId
df_unqiue_sess['customerSessionNumber'] = df_unqiue_sess.groupby('customerVisitorId').cumcount()+1

#create a reference mapping dictionary
sess_num_dict = dict(zip(df_unqiue_sess.customerSessionId, df_unqiue_sess.customerSessionNumber.values))

#fill in customerSessionNumber with mapping dictionary
df_ec_coded.customerSessionNumber = df_ec_coded.customerSessionNumber.fillna(df_ec_coded.customerSessionId.map(sess_num_dict))


In [ ]:
#transition into final dataframe
df_ec_final = df_ec_coded

### Update or Drop Column Names




In [ ]:
#check columns and datat types
df_ec_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25323266 entries, 25323265 to 0
Data columns (total 47 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   event_time                   datetime64[ns]
 1   product_id                   int64         
 2   category_id                  int64         
 3   product_brand                object        
 4   product_price                float64       
 5   customerVisitorId            int64         
 6   customerSessionId            object        
 7   unique_key                   object        
 8   unique_sess_event            object        
 9   unique_sess                  object        
 10  totals_events                float64       
 11  event_type_original          object        
 12  event_type_cart              int64         
 13  event_type_purchase          int64         
 14  event_type_remove_from_cart  int64         
 15  event_type_view              int64         
 16

In [ ]:
#rename columns
df_ec_final = df_ec_coded.rename(columns={
     'isExit': 'last_session_event',
     'isEntrance': 'first_session_event'
})

In [ ]:
#drop uneccessary columns
df_ec_final = df_ec_final.drop(columns=['unique_key', 'unique_sess_event', 'event_type_original','min','max'])

### Address Missing Values for New Featurees

In [ ]:
# check missing values to determine next teps
null_columns= df_ec_final.columns[df_ec_final.isnull().any()]
df_ec_final[null_columns].isnull().sum()

Series([], dtype: float64)

In [ ]:
#check data
df_ec_final.head()

,event_time,product_id,category_id,product_brand,product_price,customerVisitorId,customerSessionId,unique_sess,totals_events,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view,totalViewProducts,totalViewProductRevenue,totalAddToCarts,totalAddToCartRevenue,totalRemoveFromCarts,totalRemoveFromCartRevenue,totalOrders,totalOrderRevenue,uniqueViewProducts,uniqueAddToCarts,uniqueRemoveFromCarts,uniqueOrders,event_year,event_month,event_day_of_week,event_day_of_month,event_day_of_year,event_hour,event_minute,event_second,event_micorsecond,weekend,holiday,timeOnSite,newSession,first_session_event,last_session_event,hits.isEntrance,customerSessionNumber
25323265,2019-10-01,5773203,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,46324001126dd6e6e-4dac-4778-8d2c-92e149dab885,0.0,1,0,0,0,0.0,0.00,7.0,16.67,1.0,0.95,19.0,39.21,0.0,7.0,1.0,19.0,2019,10,1,1,274,0,0,0,0,0,False,60.0,1.0,1.0,0.0,1,1.0
25323045,2019-10-01,5842144,1487580006317032337,irisk,1.59,385985999,d30965e8-1101-44ab-b45d-cc1bb9fae694,385985999d30965e8-1101-44ab-b45d-cc1bb9fae694,0.0,0,0,1,0,4.0,17.72,40.0,97.86,26.0,50.18,4.0,14.89,2.0,10.0,9.0,4.0,2019,10,1,1,274,0,0,0,0,0,False,120.0,1.0,0.0,0.0,1,1.0
25323046,2019-10-01,5842144,1487580006317032337,irisk,1.59,385985999,d30965e8-1101-44ab-b45d-cc1bb9fae694,385985999d30965e8-1101-44ab-b45d-cc1bb9fae694,0.0,0,0,1,0,4.0,17.72,40.0,97.86,26.0,50.18,4.0,14.89,2.0,10.0,9.0,4.0,2019,10,1,1,274,0,0,0,0,0,False,120.0,1.0,0.0,0.0,2,1.0
25323047,2019-10-01,5773351,1487580005134238553,runail,2.62,430174032,73dea1e7-664e-43f4-8b30-d32b9d5af04f,43017403273dea1e7-664e-43f4-8b30-d32b9d5af04f,0.0,0,0,1,0,13.0,32.78,2.0,5.24,2.0,5.40,0.0,0.00,9.0,2.0,2.0,0.0,2019,10,1,1,274,0,0,0,0,0,False,60.0,1.0,0.0,0.0,1,1.0
25323048,2019-10-01,5698989,1487580006317032337,irisk,1.27,385985999,d30965e8-1101-44ab-b45d-cc1bb9fae694,385985999d30965e8-1101-44ab-b45d-cc1bb9fae694,0.0,0,0,1,0,4.0,17.72,40.0,97.86,26.0,50.18,4.0,14.89,2.0,10.0,9.0,4.0,2019,10,1,1,274,0,0,0,0,0,False,120.0,1.0,0.0,0.0,3,1.0


### Save to CSV

In [ ]:
#save to csv
df_ec_final.to_csv('df_ec_final.csv',index =False)